In [1]:
import sqlite3
import random
from pyvis.network import Network
import networkx as nx
from functools import reduce
from math import floor

In [2]:
# 打疫苗之後的感染性
iVaccInfProb = 0
# 被感染的節點顏色
hexInfColor = '#fc0303'
# 打過疫苗的節點顏色
hexVaccColor = '#7cff0a'
# 群體中想要接種的比例
floatVaccRatio = 0.4
# 預計要看結果的天數
iDays = 10 
# 上面那個比例要分幾天打完（這個數字要 <= iDays）
iVaccDays = 10
# 第一個被感染的人
iFirstInfPid = 0
# 最大可能接觸人數
listMaxConn = [1, 3, 9, 50]
# 最大可能接觸人數的權重（越大越可能被挑出來）
listWtOfChoiceMaxConn = [2, 11, 13, 5]
# 依感染天數決定感染力的字典
dictInfectivity = {1: 0, 2: 0, 3: 0, 4: 0.6, 5: 0.7, 6: 0.6, 7: 0.4, 8: 0.2, 9: 0.1, 10: 0}
# 族群母數
iPopCnt = 500
# 用來紀錄人與人關係的 list
listPR = []

In [3]:
class Person:
    def __init__(self, iPid, boolInfFlag=False, floatInfProbability=0, boolIfVacc=False, iInfDays=0):
        self.pid = iPid
        self.boolIfInf = boolInfFlag
        self.boolIfVacc = boolIfVacc
        self.floatInfProbability = floatInfProbability
        self.iInfDays=iInfDays
        
    def IsInfected(self):
        self.boolIfInf = True
        
    def IsVacced(self):
        self.boolIfInf = False
        self.boolIfVacc = True
        self.floatInfProbability = iVaccInfProb
        
    def ResetInfStatus(self):
        self.boolIfInf = False
        self.boolIfVacc=False
        self.floatInfProbability = 0
        self.iInfDays = 0

In [4]:
def getRelatePidsWithThisMan(iPid, listAllPopRels):
    listRelsWithThisPerson = list(filter(lambda x: x[0] == iPid or x[1] == iPid, listAllPopRels))
    if listRelsWithThisPerson == []:
        return []
    # 挑出跟這個人有關的人際關係
    listGrpMightBeInf = list(set(reduce(lambda x, y: x + y, listRelsWithThisPerson)))
    # 移除這個人
    listGrpMightBeInf.remove(iPid)
    return listGrpMightBeInf

def NetworkMaterialization(listAllPop, listAllPopRels, sNetworkName):
    netPRInf = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')
    for person in listAllPop:
        if person.boolIfInf == True:
            netPRInf.add_node(person.pid, title=str(person.pid), color=hexInfColor, value=10)
        elif person.boolIfVacc == True:
            netPRInf.add_node(person.pid, title=str(person.pid), color=hexVaccColor, value=10)
        else:
            netPRInf.add_node(person.pid, title=str(person.pid), value=10)
    
    for relation in listAllPopRels:
        netPRInf.add_edge(relation[0], relation[1])
    
    netPRInf.barnes_hut()
    netPRInf.show(f'{sNetworkName}.html')
    
# 把要接種疫苗的總人數依要接種的天數分批
def getListChunks(iToBeDivided, iChunkCnt):
	iStep = floor(iToBeDivided/iChunkCnt)
	listChunks = [iStep] * iChunkCnt
	listChunks[-1] += iToBeDivided - sum(listChunks)
	return listChunks
    
# 找出接觸最多人且沒打過疫苗的人群
def getListTopNGrpByContactCntNoVacc(listAllPop, listAllPopRels, iTopN=100, boolSortContactCntReverse=True):
    listPidContactCnt = []
    # 取出沒有打過疫苗的人們
    listAllNotVaccPeople = list(filter(lambda x: x.boolIfVacc is False, listAllPop))
    # 沒打過疫苗才有傳染可能
    for person in listAllNotVaccPeople:
        listMightBeInfPids = getRelatePidsWithThisMan(person.pid, listAllPopRels)
        listPidContactCnt.append((person.pid, len(listMightBeInfPids)))
    # 0 號不能打疫苗，是第一位感染者。
    listPidContactCnt = list(filter(lambda x: x[0] != 0, listPidContactCnt))
    listTopNPidsByContactCnt = sorted(listPidContactCnt, key=lambda x: x[1], reverse=boolSortContactCntReverse)[0:iTopN]
    return listTopNPidsByContactCnt

In [5]:
connPR = sqlite3.connect('./PeopleRelation.db')
cur = connPR.cursor()

In [6]:
# drop 所有表格
cur.execute('drop table if exists Relation')
cur.execute('drop table if exists People')

In [7]:
cur.execute(
'''
    create table if not exists People( pid integer not null primary key )
''')

cur.execute(
'''
    create table if not exists Relation( pid integer not null,
    pidconn integer not null,
    primary key(pid,
    pidconn),
    foreign key (pid) references People(pid),
    foreign key (pidconn) references People(pid) )
''')

In [8]:
listPeople = [Person(x) for x in range(iPopCnt)]

In [9]:
# insert data into table People
for person in listPeople:
    cur.execute(f'insert into People values({person.pid})')
connPR.commit()

In [10]:
# 模擬人與人之間的接觸關係
listPR = []
for person in listPeople:
    iRandconnCnt = random.choices(listMaxConn, listWtOfChoiceMaxConn)[0]
    # 把 person 挑掉
    listPeopleWithoutPerson = [x for x in listPeople if x.pid != person.pid]
    # 根據隨機選出的 MaxConn 數目挑出可能接觸的 pid
    listPidConnTo = random.choices(listPeopleWithoutPerson, k=iRandconnCnt)

    for PersonConnTo in listPidConnTo:
        if person.pid < PersonConnTo.pid:
            tuplePR = (person.pid, PersonConnTo.pid)
        else:
            tuplePR = (PersonConnTo.pid, person.pid)
        listPR.append(tuplePR)
listPR = list(set(listPR))

In [11]:
cur.executemany('insert into Relation values(?, ?)', listPR)
connPR.commit()

In [12]:
# 畫出未受感染的群體
NetworkMaterialization(listPeople, listPR, 'NetOrig')

In [13]:
# 未施予疫苗保護的群體
for person in listPeople:
    person.ResetInfStatus()

listPeople[iFirstInfPid].IsInfected()

for day in range(iDays):
    for person in listPeople:
        listMightBeInfPids = getRelatePidsWithThisMan(person.pid, listPR)
        
        # 如果被感染就感染天數開始加一，這個天數會影響感染機率
        if person.boolIfInf == True:
            person.iInfDays += 1
        # 如果感染機率大於 0 ，就開始感染過程。
        # 首先依照被感染天數決定感染機率
        person.floatInfProbability = dictInfectivity.get(person.iInfDays, 0)
        # print(person.pid, person.boolIfInf, person.iInfDays, person.floatInfProbability)

        if person.floatInfProbability > 0:
            # 根據這個人的感染機率隨機挑人
            listBeInfPids = random.sample(listMightBeInfPids, k=round(len(listMightBeInfPids) * person.floatInfProbability))
            for iInfPid in listBeInfPids:
                listPeople[iInfPid].IsInfected()

NetworkMaterialization(listPeople, listPR, 'NetOrigInf')

In [14]:
len(list(filter(lambda x: x.boolIfInf == False, listPeople)))

3

In [15]:
# 以接觸最多人數選擇打疫苗族群的策略

# 看需要多少接種接種比例，用這個比例算出在 iVaccDays 內每天要接種多少人的 list
# 例如有 10 人要分 3 天接種，則得到 [3, 3, 4]
# 這裡要注意，總人數乘以浮點數會變成浮點數，所以分批完以後會變成浮點數。
# 而這個數字是要拿來做 slice 的 index ，這樣會出錯。
# 所以要進行數字分批的時候要注意，輸入 getListChunks 的參數必須是整數（要 round 過）。
listVaccChunks = getListChunks(round(len(listPeople) * floatVaccRatio), iVaccDays)

# 重設 listPeople
for person in listPeople:
    person.ResetInfStatus()

# 感染第一個人
listPeople[iFirstInfPid].IsInfected()

for day in range(iDays):
    iVaccChunk = listVaccChunks.pop() if len(listVaccChunks) > 0 else 0
    listTopNPidsWithContactCntNoVacc = getListTopNGrpByContactCntNoVacc(listPeople, listPR, iTopN=iVaccChunk,
                                                                  boolSortContactCntReverse=True)
    # 挑出要打疫苗的族群 Pids
    listPidsForVacc = list(map(lambda x: x[0], listTopNPidsWithContactCntNoVacc))
    # 每天幫接觸最多人的族群前 N 名打疫苗
    for PidForVac in listPidsForVacc:
        listPeople[PidForVac].boolIfVacc = True

    # 找出有打疫苗的 people
    listPeopleVacc = list(filter(lambda x: x.boolIfVacc is True, listPeople))
    # 建立有打疫苗的 pids list
    listPidsVacc = list(map(lambda x: x.pid, listPeopleVacc))

    for person in listPeople:
        if person.boolIfVacc == True:
            continue
        # 挑出跟這個人有關係的 pids
        listMightBeInfPids = getRelatePidsWithThisMan(person.pid, listPR)
        # 從關係者清單中挖掉打過疫苗的人，剩下的就是可以感染的人
        listMightBeInfPidsNoVacc = list(filter(lambda x: x not in listPidsVacc, listMightBeInfPids))

        # 如果被感染就感染天數開始加一，這個天數會影響感染機率
        if person.boolIfInf is True:
            person.iInfDays += 1
        # 如果感染機率大於 0 ，就開始感染過程。
        # 首先依照被感染天數決定感染機率
        person.floatInfProbability = dictInfectivity.get(person.iInfDays, 0)

        if person.floatInfProbability > 0:
            # 根據這個人的感染機率隨機挑人
            listBeInfPids = random.sample(listMightBeInfPidsNoVacc,
                                          k=round(len(listMightBeInfPidsNoVacc) * person.floatInfProbability))
            for iInfPid in listBeInfPids:
                listPeople[iInfPid].IsInfected()
                    
NetworkMaterialization(listPeople, listPR, 'NetVaccByContactCntDesc')

In [16]:
len(list(filter(lambda x: x.boolIfInf == False, listPeople)))

382

In [17]:
# 以接觸最多人數選擇打疫苗族群的策略

# 看需要多少接種接種比例，用這個比例算出在 iVaccDays 內每天要接種多少人的 list
# 例如有 10 人要分 3 天接種，則得到 [3, 3, 4]
# 這裡要注意，總人數乘以浮點數會變成浮點數，所以分批完以後會變成浮點數。
# 而這個數字是要拿來做 slice 的 index ，這樣會出錯。
# 所以要進行數字分批的時候要注意，輸入 getListChunks 的參數必須是整數（要 round 過）。
listVaccChunks = getListChunks(round(len(listPeople) * floatVaccRatio), iVaccDays)

# 重設 listPeople
for person in listPeople:
    person.ResetInfStatus()

# 感染第一個人
listPeople[iFirstInfPid].IsInfected()

for day in range(iDays):
    iVaccChunk = listVaccChunks.pop() if len(listVaccChunks) > 0 else 0
    listTopNPidsWithContactCntNoVacc = getListTopNGrpByContactCntNoVacc(listPeople, listPR, iTopN=iVaccChunk,
                                                                  boolSortContactCntReverse=False)
    # 挑出要打疫苗的族群 Pids
    listPidsForVacc = list(map(lambda x: x[0], listTopNPidsWithContactCntNoVacc))
    # 每天幫接觸最多人的族群前 N 名打疫苗
    for PidForVac in listPidsForVacc:
        listPeople[PidForVac].boolIfVacc = True

    # 找出有打疫苗的 people
    listPeopleVacc = list(filter(lambda x: x.boolIfVacc is True, listPeople))
    # 建立有打疫苗的 pids list
    listPidsVacc = list(map(lambda x: x.pid, listPeopleVacc))

    for person in listPeople:
        if person.boolIfVacc == True:
            continue
        # 挑出跟這個人有關係的 pids
        listMightBeInfPids = getRelatePidsWithThisMan(person.pid, listPR)
        # 從關係者清單中挖掉打過疫苗的人，剩下的就是可以感染的人
        listMightBeInfPidsNoVacc = list(filter(lambda x: x not in listPidsVacc, listMightBeInfPids))

        # 如果被感染就感染天數開始加一，這個天數會影響感染機率
        if person.boolIfInf is True:
            person.iInfDays += 1
        # 如果感染機率大於 0 ，就開始感染過程。
        # 首先依照被感染天數決定感染機率
        person.floatInfProbability = dictInfectivity.get(person.iInfDays, 0)

        if person.floatInfProbability > 0:
            # 根據這個人的感染機率隨機挑人
            listBeInfPids = random.sample(listMightBeInfPidsNoVacc,
                                          k=round(len(listMightBeInfPidsNoVacc) * person.floatInfProbability))
            for iInfPid in listBeInfPids:
                listPeople[iInfPid].IsInfected()
                    
NetworkMaterialization(listPeople, listPR, 'NetVaccByContactCntAsc')

In [18]:
len(list(filter(lambda x: x.boolIfInf == False, listPeople)))

173

In [19]:
connPR.close()